In [1]:
import pandas as pd
import re

# carrega tudo
df = pd.read_csv("cenarioB.csv")

# Filtrar apenas as linhas do tipo "vector"
df_vectors = df[df['type'] == 'vector'].copy()

# exibe só as 5 primeiras linhas de vetores
df_vectors.head()

,run,type,module,name,attrname,attrvalue,vectime,vecvalue
168,VoIP-UL-0-20250707-14:40:08-1607763,vector,Highway5G.server.ppp[0].queue,queueLength:vector,NaN,NaN,0 0 0 0 0,0 0 0 0 0
175,VoIP-UL-0-20250707-14:40:08-1607763,vector,Highway5G.server.ppp[0].queue,queueBitLength:vector,NaN,NaN,0 0 0 0 0,0 0 0 0 0
182,VoIP-UL-0-20250707-14:40:08-1607763,vector,Highway5G.server.ppp[0].ppp,transmissionState:vector,NaN,NaN,0,0
187,VoIP-UL-0-20250707-14:40:08-1607763,vector,Highway5G.server.udp,passedUpPk:vector(count),NaN,NaN,0 1.5090003588 1.5280003588 1.5480003588 1.568...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
192,VoIP-UL-0-20250707-14:40:08-1607763,vector,Highway5G.server.udp,droppedPkWrongPort:vector(count),NaN,NaN,0,0


In [2]:
def build_df(param, column_name):
    item_df = df_vectors[df_vectors['name'].str.contains(param, case=False, na=False)]

    timestamps = []
    items = []
    cars = []

    for _, row in item_df.iterrows():
        # separa valores de tempo e valor do vetor
        times = str(row['vectime']).split()
        values = str(row['vecvalue']).split()
        
        # extrai número do carro do módulo
        match = re.search(r'car\[(\d+)\]', row['module'])
        car_number = int(match.group(1)) if match else None
        
        for t, v in zip(times, values):
            timestamps.append(float(t))
            items.append(float(v))
            cars.append(car_number)

    # monta o dataframe final
    partial_df = pd.DataFrame({
        "timestamp": timestamps,
        column_name: items,
        "car": cars
    })

    # ordena por timestamp
    partial_df = partial_df.sort_values(by="timestamp").reset_index(drop=True)
    partial_df = (
        partial_df.groupby(['timestamp', 'car'])
        .first()
        .reset_index()
    )

    return partial_df

In [3]:
serving_cell_df = build_df("servingCell", "serving_cell")
mac_delay_ul_df = build_df("macDelayUL", "mac_delay_ul")
mac_delay_dl_df = build_df("macDelayUL", "mac_delay_dl")
average_cqi_ul_df = build_df("averageCqiDL", "average_cqi_ul")
average_cqi_dl_df = build_df("averageCqiUL", "average_cqi_dl")
throughput_df = build_df("voIPGeneratedThroughput", "throughput")

In [4]:
def merge_dfs(param, final_df, new_df):
    params = {
        "mac_delay_ul": "zero",
        "mac_delay_dl": "zero",
        "average_cqi_ul": "ffill",
        "average_cqi_dl": "ffill",
        "throughput": "zero"
    }

    final_df = pd.merge(
        final_df,
        new_df,
        on=["timestamp", "car"],
        how="outer"
    )

    # regra de preenchimento
    if params[param] == "zero":
        final_df[param] = final_df[param].fillna(0)
    elif params[param] == "ffill":
        final_df[param] = final_df.groupby("car")[param].ffill()
        # se ainda existir NaN no início (não tem valor antes)
        final_df[param] = final_df[param].fillna(0)
    
    return final_df

In [5]:
# 1) Começa do serving_cell como referência
final_df = serving_cell_df.copy()

# 2) Lista dos demais dataframes com as respectivas regras
# form: (dataframe, nome da coluna, regra de merge)
parametros = [
    (mac_delay_ul_df, "mac_delay_ul", "zero"),
    (mac_delay_dl_df, "mac_delay_dl", "zero"),
    (average_cqi_ul_df, "average_cqi_ul", "ffill"),
    (average_cqi_dl_df, "average_cqi_dl", "ffill"),
    (throughput_df, "throughput", "zero"),
]

In [6]:
final_df = merge_dfs("mac_delay_ul", final_df, mac_delay_ul_df)
final_df.shape

(24005, 4)

In [7]:
final_df = merge_dfs("mac_delay_dl", final_df, mac_delay_dl_df)
final_df.shape

(24005, 5)

In [8]:
final_df = merge_dfs("average_cqi_ul", final_df, average_cqi_ul_df)
final_df.shape

(24005, 6)

In [9]:
final_df = merge_dfs("average_cqi_dl", final_df, average_cqi_dl_df)
final_df.shape

(68231, 7)

In [10]:
final_df = merge_dfs("throughput", final_df, throughput_df)
final_df.shape

(88885, 8)

In [11]:
final_df["serving_cell"] = final_df.groupby("car")["serving_cell"].ffill()
final_df["serving_cell"] = final_df["serving_cell"].fillna(0)

In [12]:
for col in ["average_cqi_ul", "average_cqi_dl"]:
    final_df[col] = final_df.groupby("car")[col].ffill().fillna(0)

In [13]:
final_df = final_df.sort_values(["car", "timestamp"]).reset_index(drop=True)
final_df.head()

,timestamp,car,serving_cell,mac_delay_ul,mac_delay_dl,average_cqi_ul,average_cqi_dl,throughput
0,1.500,0,0.0,0.000,0.000,0.0,0.0,26.666667
1,1.505,0,0.0,NaN,NaN,0.0,14.0,0.000000
2,1.509,0,0.0,0.004,0.004,0.0,14.0,0.000000
3,1.520,0,0.0,NaN,NaN,0.0,14.0,52.631579
4,1.524,0,0.0,NaN,NaN,0.0,14.0,0.000000


In [14]:
# vamos inicializar a coluna de handover
final_df["handover"] = 0

# percorre cada carro separadamente
for car_id, group in final_df.groupby("car"):
    last_valid_cell = None
    
    for idx, row in group.iterrows():
        current_cell = row["serving_cell"]
        
        # ignora 0
        if current_cell == 0:
            continue
        
        # se houve troca de servingCell real
        if last_valid_cell is None:
            last_valid_cell = current_cell
        elif current_cell != last_valid_cell:
            # houve mudança (handover)
            final_df.loc[idx, "handover"] = 1
            last_valid_cell = current_cell


In [15]:
final_df.head()

,timestamp,car,serving_cell,mac_delay_ul,mac_delay_dl,average_cqi_ul,average_cqi_dl,throughput,handover
0,1.500,0,0.0,0.000,0.000,0.0,0.0,26.666667,0
1,1.505,0,0.0,NaN,NaN,0.0,14.0,0.000000,0
2,1.509,0,0.0,0.004,0.004,0.0,14.0,0.000000,0
3,1.520,0,0.0,NaN,NaN,0.0,14.0,52.631579,0
4,1.524,0,0.0,NaN,NaN,0.0,14.0,0.000000,0


In [16]:
positions = pd.read_csv("positionsB.csv")
positions["car"] = positions["vehicle_id"].str.extract(r"(\d+)").astype(int)
positions.rename(columns={"x": "pos_x", "y": "pos_y"}, inplace=True)
positions = positions.sort_values(["car", "time"]).reset_index(drop=True)
final_df = final_df.sort_values(["car", "timestamp"]).reset_index(drop=True)

print(positions.head())

   time vehicle_id    pos_x    pos_y  speed   angle          lane  car
0   1.0       veh0  1840.46  8646.03  14.72  151.19  -858877187_0    0
1   2.0       veh0  1847.48  8633.26  14.58  151.19  -858877187_0    0
2   3.0       veh0  1854.22  8621.01  13.98  151.19  -858877187_0    0
3   4.0       veh0  1862.14  8609.46  14.13  135.18  -858877187_0    0
4   5.0       veh0  1873.60  8601.24  14.20  120.63  -858877187_0    0


In [17]:
import numpy as np

# build index para pesquisa rápida

def pegar_pos_mais_proxima(row):
    car_id = row["car"]
    timestamp = row["timestamp"]
    
    g = positions[positions["car"] == car_id]
    
    delta = 1e-6
    anteriores = g[g["time"] <= (timestamp + delta)]
    if anteriores.empty:
        return pd.Series([np.nan, np.nan, np.nan, np.nan])
    
    linha = anteriores.iloc[-1]
    return pd.Series([linha["pos_x"], linha["pos_y"], linha["speed"], linha["angle"]])


# aplicar
final_df[["pos_x","pos_y","speed","angle"]] = final_df.apply(pegar_pos_mais_proxima, axis=1)

In [18]:
print(final_df.head())

   timestamp  car  serving_cell  mac_delay_ul  mac_delay_dl  average_cqi_ul  \
0      1.500    0           0.0         0.000         0.000             0.0   
1      1.505    0           0.0           NaN           NaN             0.0   
2      1.509    0           0.0         0.004         0.004             0.0   
3      1.520    0           0.0           NaN           NaN             0.0   
4      1.524    0           0.0           NaN           NaN             0.0   

   average_cqi_dl  throughput  handover    pos_x    pos_y  speed   angle  
0             0.0   26.666667         0  1840.46  8646.03  14.72  151.19  
1            14.0    0.000000         0  1840.46  8646.03  14.72  151.19  
2            14.0    0.000000         0  1840.46  8646.03  14.72  151.19  
3            14.0   52.631579         0  1840.46  8646.03  14.72  151.19  
4            14.0    0.000000         0  1840.46  8646.03  14.72  151.19  


In [19]:
final_df["next_serving_cell"] = final_df.groupby("car")["serving_cell"].shift(-1).fillna(0).astype(int)

In [20]:
final_df["delta_x"] = final_df.groupby("car")["pos_x"].diff().fillna(0)
final_df["delta_y"] = final_df.groupby("car")["pos_y"].diff().fillna(0)

In [21]:
final_df["accel"] = final_df.groupby("car")["speed"].diff().fillna(0)

In [22]:
final_df["mac_delay_ul"] = final_df["mac_delay_ul"].fillna(0)
final_df["mac_delay_dl"] = final_df["mac_delay_dl"].fillna(0)

In [23]:
final_df["scenario"] = "B"  # para o bairro

In [24]:
print(final_df.head(10))

   timestamp  car  serving_cell  mac_delay_ul  mac_delay_dl  average_cqi_ul  \
0      1.500    0           0.0         0.000         0.000             0.0   
1      1.505    0           0.0         0.000         0.000             0.0   
2      1.509    0           0.0         0.004         0.004             0.0   
3      1.520    0           0.0         0.000         0.000             0.0   
4      1.524    0           0.0         0.000         0.000             0.0   
5      1.528    0           0.0         0.004         0.004             0.0   
6      1.540    0           0.0         0.000         0.000             0.0   
7      1.544    0           0.0         0.000         0.000             0.0   
8      1.548    0           0.0         0.004         0.004             0.0   
9      1.560    0           0.0         0.000         0.000             0.0   

   average_cqi_dl  throughput  handover    pos_x    pos_y  speed   angle  \
0             0.0   26.666667         0  1840.46  8646

In [25]:
final_df.to_csv("handover_dataset_B.csv", index=False)

In [29]:
import pandas as pd

# Carrega os três arquivos
df1 = pd.read_csv("handover_dataset_A.csv")
df2 = pd.read_csv("handover_dataset_B.csv")
df3 = pd.read_csv("handover_dataset_C.csv")

# Concatena todos em um único DataFrame
df_total = pd.concat([df1, df2, df3], ignore_index=True)

# Verifica rapidamente o resultado
print(df_total["scenario"].value_counts())
print(df_total.shape)

df_total.to_csv("handover_dataset.csv", index=False)

scenario
A    429710
C    368891
B     88885
Name: count, dtype: int64
(887486, 18)
